# Ensemble Learning

## Initial Imports

In [66]:
import warnings
warnings.filterwarnings('ignore')

In [67]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [68]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [69]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [70]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

le.fit(df['home_ownership'])
df['home_ownership'] = le.transform(df['home_ownership'])

le.fit(df['verification_status'])
df['verification_status'] = le.transform(df['verification_status'])

le.fit(df['loan_status'])
df['loan_status'] = le.transform(df['loan_status'])

le.fit(df['pymnt_plan'])
df['pymnt_plan'] = le.transform(df['pymnt_plan'])

df

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,3,66000.0,1,Mar-2019,1,0,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,1,105000.0,2,Mar-2019,1,0,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,1,56000.0,2,Mar-2019,1,0,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,3,92000.0,2,Mar-2019,1,0,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,1,52000.0,0,Mar-2019,1,0,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68812,10000.0,0.1502,346.76,3,26000.0,1,Jan-2019,1,0,9.60,...,80.0,0.0,0.0,0.0,20625.0,6798.0,11300.0,5425.0,N,N
68813,12000.0,0.2727,368.37,3,63000.0,0,Jan-2019,1,0,29.07,...,96.2,0.0,0.0,0.0,87939.0,60350.0,13500.0,62939.0,N,N
68814,5000.0,0.1992,185.62,1,52000.0,1,Jan-2019,1,0,14.86,...,100.0,0.0,1.0,0.0,30592.0,18611.0,3600.0,18492.0,N,N
68815,40000.0,0.0646,1225.24,1,520000.0,2,Jan-2019,1,0,9.96,...,98.2,12.5,0.0,0.0,1033574.0,95958.0,100800.0,78634.0,N,N


## Split the Data into Training and Testing

In [71]:
# Create our features
X = df[["loan_amnt", "int_rate", "home_ownership", "annual_inc", "installment", "verification_status"]]

# Create our target
y = df['loan_status']

In [72]:
X.describe()

,loan_amnt,int_rate,home_ownership,annual_inc,installment,verification_status
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000
mean,16677.594562,0.127718,1.812779,8.821371e+04,480.652863,0.669994
std,10277.348590,0.048130,0.941313,1.155800e+05,288.062432,0.719105
min,1000.000000,0.060000,0.000000,4.000000e+01,30.890000,0.000000
25%,9000.000000,0.088100,1.000000,5.000000e+04,265.730000,0.000000
50%,15000.000000,0.118000,1.000000,7.300000e+04,404.560000,1.000000
75%,24000.000000,0.155700,3.000000,1.040000e+05,648.100000,1.000000
max,40000.000000,0.308400,3.000000,8.797500e+06,1676.230000,2.000000


In [73]:
# Check the balance of our target values
y.describe()

count    68817.000000
mean         0.994958
std          0.070831
min          0.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: loan_status, dtype: float64

In [74]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [75]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [76]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
x_scaler = scaler.fit(X_train)

In [77]:
# Scale the training and testing data
x_train_scaled = x_scaler.transform(X_train)
x_test_scaled = x_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [78]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=10, random_state=1)
brf.fit(X_train, y_train)

BalancedRandomForestClassifier(n_estimators=10, random_state=1)

In [79]:
# Calculated the balanced accuracy score
y_pred = brf.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.5229608730283692

In [80]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[  59,   42],
       [9206, 7898]])

In [81]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.58      0.46      0.01      0.52      0.27       101
          1       0.99      0.46      0.58      0.63      0.52      0.27     17104

avg / total       0.99      0.46      0.58      0.63      0.52      0.27     17205



In [82]:
# List the features sorted in descending order by feature importance


### Easy Ensemble Classifier

In [83]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier
eec = EasyEnsembleClassifier(n_estimators=10, random_state=1)
eec.fit(X_train, y_train)

EasyEnsembleClassifier(random_state=1)

In [84]:
# Calculated the balanced accuracy score
y_pred = eec.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.5938023877224018

In [85]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   60,    41],
       [ 6952, 10152]])

In [86]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.59      0.59      0.02      0.59      0.35       101
          1       1.00      0.59      0.59      0.74      0.59      0.35     17104

avg / total       0.99      0.59      0.59      0.74      0.59      0.35     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    YOUR ANSWER HERE.

2. Which model had the best recall score?

    YOUR ANSWER HERE.

3. Which model had the best geometric mean score?

    YOUR ANSWER HERE.

4. What are the top three features?

    YOUR ANSWER HERE.